In [1]:
import pandas as pd
import numpy as np
import re
import json
from konlpy.tag import Okt
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

###### 모델 불러오기 ######

loaded_model = load_model('./test-model1.h5')

###########################

okt = Okt()
tokenizer = Tokenizer()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다','부터']
max_len = 30
vocab_size = 10000
# tokenizer = Tokenizer(vocab_size)


#############################
# tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
# tokenizer.fit_on_texts(new_sentence)

###### json 파일 불러오기 ######

with open('../일베집합_test4.json', encoding='utf-8') as json_file:
    vocab = json.load(json_file)
    tokenizer.word_index = vocab
    
################################

def slang_predict(new_sentence):
#     new_sentence = []
#     tokenizer = Tokenizer(vocab_size, oov_token='OOV') #OOV 토큰
#     tokenizer.fit_on_texts(new_sentence)
#     train_input = tokenizer.texts_to_sequences(new_sentence)
#     word_index = tokenizer.word_index(new_sentence)
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', str(new_sentence))
    if new_sentence == "" or new_sentence.isspace():
        return '일반 댓글'
    else : 
        new_sentence = okt.morphs(new_sentence, stem=True)
        new_sentence = [word for word in new_sentence if not word in stopwords]
        if not new_sentence:
            return '일반 댓글'
        else:
            encoded = tokenizer.texts_to_sequences([new_sentence])
            pad_new = pad_sequences(encoded, maxlen = max_len)
            score = loaded_model.predict(pad_new)
            if(score > 0.7): 
                return "{0}, 비속어 포함".format(round(float(score), 2))
            else:
                return "{0}, 일반 댓글".format(round(float(1 - score), 2))

# word_to_index = {}
# word_to_index['OOV'] = len(word_to_index) + 1
# print(word_to_index)
# encoded_sentences = []

# for sentence in preprocessed_sentences:
#     encoded_sentence = []
#     for word in sentence:
#         try:
#             # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
#             encoded_sentence.append(word_to_index[word])
#         except KeyError:
#             # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
#             encoded_sentence.append(word_to_index['OOV'])
#     encoded_sentences.append(encoded_sentence)


2022-05-25 05:55:01.121313: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-25 05:55:01.152240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-05-25 05:55:01.152412: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-05-25 05:55:01.153797: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-25 05:55:01.155246: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-05-25 05:55:01.155481: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-05-

In [2]:
slang_predict("근데 저게 이뻐보이나? 저 가격이면 쿠팡에서 두개 사고도 남음 ")

InvalidArgumentError:  indices[0,24] = 6814 is not in [0, 2908)
	 [[node sequential_9/embedding_9/embedding_lookup (defined at tmp/ipykernel_31357/1176218959.py:58) ]] [Op:__inference_predict_function_1162]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_9/embedding_9/embedding_lookup:
 sequential_9/embedding_9/embedding_lookup/707 (defined at anaconda/envs/check_up_py38_PT_TF/lib/python3.8/contextlib.py:113)

Function call stack:
predict_function


In [58]:
slang_predict("와 가격 미친거 아님? ㅠㅠ 남는게 있나요")

'0.99, 일반 댓글'

In [3]:
slang_predict("ㅋㅋㅋ 미친 저게 뭐야 ㅠ")

'0.82, 비속어 포함'

In [60]:
slang_predict("파란색 존나 이쁘다")

'0.98, 일반 댓글'

In [4]:
slang_predict("존나 가 들어가면 욕설이라고 나와야하는데..")

'0.77, 비속어 포함'

In [5]:
slang_predict("긴 댓글을 치면 비속어 분류 모델이 제대로 분류를 못하는것 같아요 ~")

InvalidArgumentError:  indices[0,25] = 9820 is not in [0, 2908)
	 [[node sequential_9/embedding_9/embedding_lookup (defined at tmp/ipykernel_31357/1176218959.py:58) ]] [Op:__inference_predict_function_1162]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_9/embedding_9/embedding_lookup:
 sequential_9/embedding_9/embedding_lookup/707 (defined at anaconda/envs/check_up_py38_PT_TF/lib/python3.8/contextlib.py:113)

Function call stack:
predict_function


## OOV 예제코드 해보기

In [35]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.preprocessing.text import Tokenizer

# sentences = [
#     'I love my dog',
#     'I love my cat',
#     'You love my dog!'
# ]

# tokenzier = Tokenizer(num_words = 100) #최대 빈도수 기준으로 100개까지만 사용
# tokenizer.fit_on_texts(sentences)
# word_index = tokenizer.word_index # 단어 : 토큰번호 딕셔너리 반환
# print(word_index)

{'<OOV>': 1, 'love': 2, 'my': 3, 'i': 4, 'dog': 5, 'cat': 6, 'you': 7}


In [36]:
# from tensorflow.keras.preprocessing.text import Tokenizer

# sentence = [
#     'I love my dog',
#     'I love my cat',
#     'You love my dog!',
#     'Do you think my dog is amazing?'
# ]

# tokenizer = Tokenizer(num_words= 100) #얘를 바꾸는거!
# tokenizer.fit_on_texts(sentences)
# word_index = tokenzier.word_index

# sequences = tokenzier.texts_to_sequences(sentences)

# print(word_index)
# print(sequences)

{}
[[], [], []]


In [37]:
# tokenizer = Tokenizer(num_words = 100, oov_token = "<OOV>")

In [38]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# padded = pad_sequences(sequences, padding = 'post', maxlen = 5)

In [39]:
# import tensorflow as tf
# from tensorflow import keras


# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# sentences = [
#     'I love my dog',
#     'I love my cat',
#     'You love my dog!',
#     'Do you think my dog is amazing?'
# ]

# tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
# tokenizer.fit_on_texts(sentences)
# word_index = tokenizer.word_index

# sequences = tokenizer.texts_to_sequences(sentences)

# padded = pad_sequences(sequences, maxlen=5)
# print("\nWord Index = " , word_index)
# print("\nSequences = " , sequences)
# print("\nPadded Sequences:")
# print(padded)


# # Try with words that the tokenizer wasn't fit to
# test_data = [
#     'i really love my dog',
#     'my dog loves my manatee'
# ]

# test_seq = tokenizer.texts_to_sequences(test_data)
# print("\nTest Sequence = ", test_seq)

# padded = pad_sequences(test_seq, maxlen=10)
# print("\nPadded Test Sequence: ")
# print(padded)


Word Index =  {'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}

Sequences =  [[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]

Padded Sequences:
[[ 0  5  3  2  4]
 [ 0  5  3  2  7]
 [ 0  6  3  2  4]
 [ 9  2  4 10 11]]

Test Sequence =  [[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]]

Padded Test Sequence: 
[[0 0 0 0 0 5 1 3 2 4]
 [0 0 0 0 0 2 4 1 2 1]]


In [40]:
# import json

# with open("./일베집합test3.json", 'r') as f:
#     datastore = json.load(f)


# sentences = [] 
# for item in datastore:
#     sentences.append(item['headline'])
#     labels.append(item['is_sarcastic'])
#     urls.append(item['article_link'])



# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# tokenizer = Tokenizer(oov_token="<OOV>")
# tokenizer.fit_on_texts(sentences)

# word_index = tokenizer.word_index
# print(len(word_index))
# print(word_index)
# sequences = tokenizer.texts_to_sequences(sentences)
# padded = pad_sequences(sequences, padding='post')
# print(padded[0])
# print(padded.shape)

TypeError: string indices must be integers

In [42]:

# with open('../일베집합_test4.json', encoding='utf-8') as json_file:
#     vocab = json.load(json_file)
#     tokenizer.word_index = vocab

In [43]:
# tokenizer = Tokenizer(vocab_size, oov_token='OOV') #OOV 토큰
# tokenizer.fit_on_texts(new_sentence)
# train_input = tokenizer.texts_to_sequences(new_sentence)

NameError: name 'clean_train_reviews' is not defined

In [41]:
# word_to_index = {}
# i = 0
# for (word, frequency) in vocab_sorted :
#     if frequency > 1 : # 빈도수가 작은 단어는 제외.
#         i = i + 1
#         word_to_index[word] = i

# print(word_to_index)

NameError: name 'vocab_sorted' is not defined

In [ ]:
# tokenizer = Tokenizer(num_words = vocab_size, oov_token="<OOV>")
# tokenizer.fit_on_texts(sentences)
# word_index = tokenizer.word_index

In [ ]:
# word_to_index = {}
# word_to_index['OOV'] = len(word_to_index) + 1
# print(word_to_index)
# encoded_sentences = []

# for sentence in preprocessed_sentences:
#     encoded_sentence = []
#     for word in sentence:
#         try:
#             # 단어 집합에 있는 단어라면 해당 단어의 정수를 리턴.
#             encoded_sentence.append(word_to_index[word])
#         except KeyError:
#             # 만약 단어 집합에 없는 단어라면 'OOV'의 정수를 리턴.
#             encoded_sentence.append(word_to_index['OOV'])
#     encoded_sentences.append(encoded_sentence)
